In [102]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [121]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=359):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [122]:
# def get_naver_articles(urls):
#     articles, titles, dates, press = [], [], [], []
    
#     for url in urls:
#         html = requests.get(url, headers=headers)
#         soup = BeautifulSoup(html.text, "html.parser")

#         article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
#         article_titles = [title.text for title in soup.select(".news_tit")]
#         article_infos = [info.text for info in soup.select(".info")]

#         new_urls = []
#         idx = 0

#         while idx < len(article_links) - 1:
#             if 'news.naver.com' in article_links[idx]:
#                 # Check if the next link also contains 'news.naver.com'
#                 if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
#                     new_urls.append(article_links[idx + 1])
#                     idx += 2
#                 else:
#                     new_urls.append(article_links[idx])
#                     idx += 1
#             elif 'news.naver.com' in article_links[idx + 1]:
#                 new_urls.append(article_links[idx + 1])
#                 idx += 2
#             else:
#                 new_urls.append(article_links[idx])
#                 idx += 1

#         # 마지막 요소 처리
#         if idx == len(article_links) - 1:
#             new_urls.append(article_links[idx])

#         date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

#         current_dates = []
#         current_press = []

#         for idx, info in enumerate(article_infos):
#             match = date_pattern.search(info)
#             if match:
#                 current_dates.append(match.group(0))
                
#                 if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
#                     current_press.append(article_infos[idx - 1])

#         articles.extend(new_urls)
#         titles.extend(article_titles)
#         dates.extend(current_dates)
#         press.extend(current_press)

#         time.sleep(random.uniform(1, 1.5))
        
#     return articles, titles, dates, press

In [123]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles_temp = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        # 각 페이지에서 URL이 10개가 아닌 경우 해당 페이지의 URL 출력
        if len(new_urls) % 10 != 0:
            print(f"Warning: {url} contains {len(new_urls)} links instead of 10.")
        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []
        current_titles = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press_name = article_infos[idx - 1]
                    
                    if "CHIEF EXECUTIVE" not in current_press_name:
                        current_press.append(current_press_name)
                        current_titles.append(article_titles_temp[len(current_dates)])
                        current_dates.append(match.group(0))

        articles.extend(new_urls[:len(current_titles)])  # articles length should match with titles
        titles.extend(current_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(1, 1.5))
        
    return articles, titles, dates, press

In [124]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time

In [125]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [126]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")



100%|██████████| 3583/3583 [07:56<00:00,  7.52it/s]

Length of article_urls: 3583
Length of article_titles: 3583
Length of article_dates: 3583
Length of article_press: 3583
Length of contents: 3583
Length of article_time: 3583
            date                 time  \
0     2021.06.11                        
1     2021.06.11                        
2     2021.06.11                        
3     2021.06.11  2021-06-11 00:03:19   
4     2021.06.11                        
...          ...                  ...   
3563  2021.06.22  2021-06-22 23:39:31   
3564  2021.06.22                        
3565  2021.06.22  2021-06-22 23:45:01   
3566  2021.06.22                        
3567  2021.06.22                        

                                                  title  \
0     뉴욕증시 비트코인 휘청, CPI 물가지수 "무려 5%" 인플레 폭발 연준 FOMC ...   
1     [뉴스팩트] "족쇄벗냐 vs 발목잡냐" 공수처, 尹 전 총장 '직권남용 혐의' 수사...   
2         엘살바도르 비트코인 합법적 통화 승인…Q&A로 풀어보는 암호 화폐 시장에 미...   
3                        엘살바도르 법정화폐 됐어도, 중국은 비트코인 단속 강화   
4         도지코인, 뉴욕 브루클린 플랫 부시 거리 의류‧신발

In [127]:
df

,date,time,title,content,press,link
0,2021.06.11,,"뉴욕증시 비트코인 휘청, CPI 물가지수 ""무려 5%"" 인플레 폭발 연준 FOMC ...",,글로벌이코노믹,http://www.g-enews.com/
1,2021.06.11,,"[뉴스팩트] ""족쇄벗냐 vs 발목잡냐"" 공수처, 尹 전 총장 '직권남용 혐의' 수사...",,청년일보,http://www.youthdaily.co.kr/
2,2021.06.11,,엘살바도르 비트코인 합법적 통화 승인…Q&A로 풀어보는 암호 화폐 시장에 미...,,글로벌이코노믹,http://www.g-enews.com/
3,2021.06.11,2021-06-11 00:03:19,"엘살바도르 법정화폐 됐어도, 중국은 비트코인 단속 강화","\n비트코인 하룻새 5000달러 올라중국, 해외거래소 검색차단 조치개인 투자자 우회...",E5면 1단,https://n.news.naver.com/mnews/article/025/000...
4,2021.06.11,,"도지코인, 뉴욕 브루클린 플랫 부시 거리 의류‧신발 매장 진출 외연 확대 본...",,글로벌이코노믹,http://www.g-enews.com/
...,...,...,...,...,...,...
3563,2021.06.22,2021-06-22 23:39:31,"비트코인, 1월 이후 첫 3만 달러선 붕괴…연간 마이너스 전환",\n\n\n\n\n자료사진대표적인 가상화폐 비트코인 가격이 올해 1월 말 이후 처음...,MBC언론사 선정,https://n.news.naver.com/mnews/article/214/000...
3564,2021.06.22,,"비트코인, 3만달러 하회…1월 이후 처음",,연합인포맥스,http://www.einfomax.com
3565,2021.06.22,2021-06-22 23:45:01,비트코인 급락…3만달러 깨졌다,"\n중국發 악재에 코인시장 패닉 中규제 영향…5개월 만에 최저""2만달러까지 직행 가...",A1면 1단,https://n.news.naver.com/mnews/article/015/000...
3566,2021.06.22,,일론 머스크 변덕 비트코인 가격은?...도지코인 블록크기 확대 수수료 100배 ...,,포인트데일리,http://www.thekpm.com/


In [ ]:
# end_date = '2017.10.09'

In [128]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)

In [22]:
article_urls[390:400]

['https://n.news.naver.com/mnews/article/015/0004426402?sid=102',
 'http://www.thepublic.kr/',
 'http://www.dailysecu.com',
 'http://coinreaders.com',
 'https://n.news.naver.com/mnews/article/030/0002906028?sid=105',
 'http://www.epnc.co.kr',
 'http://www.thepublic.kr/',
 'http://www.ciokorea.com',
 'http://www.breaknews.com',
 'http://www.g-enews.com/']

In [ ]:
article_urls.insert(394, "https://www.chiefexe.com/news/ArticleView.asp?listId=Mjc0NHx8bGltaXRfZmFsc2Ug")

In [52]:
df3 = df

In [56]:
df3

,date,time,title,content,press,link
0,2021.08.10,,"뉴욕증시 비트코인 휘청, CPI 물가지수 억제선 돌파 FOMC 테이퍼링 금리인상 ...",,글로벌이코노믹,http://www.g-enews.com/
1,2021.08.10,,디지털 화폐(CBDC)에 움직인 ‘8만 전자’,,스카이데일리,http://www.skyedaily.com
2,2021.08.10,,"비트코인, 한때 4만6천달러대 회복…5월 이후 최고",,연합인포맥스,http://www.einfomax.com
3,2021.08.10,2021-08-10 03:03:12,재택근무중 무심코 클릭했다 낭패… 개인PC 노린 ‘랜섬웨어 해킹’ 기승,\n코로나 이후 랜섬웨어 피해 급증세해외서버 경유땐 추적 쉽지 않아\n\n\n\n동...,A12면 1단,https://n.news.naver.com/mnews/article/020/000...
4,2021.08.10,2021-08-10 03:05:00,약탈적 '플랫폼' 금융 [우보세],\n\t\t\t[편집자주] 뉴스현장에는 희로애락이 있습니다. 그 가운데 기사로 쓰기...,9면 1단,https://n.news.naver.com/mnews/article/008/000...
...,...,...,...,...,...,...
3356,2021.08.31,,"[글로벌 규제 동향] 슬로베니아, 암호화폐 사용·판매에 10% 세금 부과 추진 ...",,코인리더스,http://coinreaders.com
3357,2021.08.31,,"엘살바도르 의회, 1.5억달러 규모 비트코인 신탁 승인...""달러와의 태환성 보...",,코인리더스,http://coinreaders.com
3358,2021.08.31,,비트코인 4만8000달러 아래서 고전하는 가운데 도지코인 50일 이동평균 수준 ...,,글로벌이코노믹,http://www.g-enews.com/
3359,2021.08.31,,"기관투자자들의 GBTC 매입 본격화?...""GBTC 락업 해제 BTC 급락 우려, ...",,코인리더스,http://coinreaders.com


In [18]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3))

NameError: name 'df2' is not defined

In [74]:
result

,date,time,title,content,press,link
0,2019.03.01,2019-03-01 00:01:01,"[가상화폐 뉴스] 03월 01일 00시 00분 비트코인(0.14%), 이오스(2.0...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1,2019.03.01,,"암호화페(가상화폐)대표 종목 비트코인시황 상승세,3월 1일 00시 기준 '비트코...",,내외경제tv,http://www.nbntv.co.kr
2,2019.03.01,,비트코인 -0.02%↓ 이더리움 0.40%↑ 리플 0.58%↑ 퀀텀 -1.30%↓ ...,,로이슈,http://www.lawissue.co.kr
3,2019.03.01,,비트코인 0.07%↑ 이더리움 0.40%↑ 리플 0% 라이트코인 0.40%↑ 대시 ...,,로이슈,http://www.lawissue.co.kr
4,2019.03.01,,비트코인 0.09%↑ 이더리움 0.40%↑ 리플 0.58%↑ 대시 -0.04%↓ 에...,,로이슈,http://www.lawissue.co.kr
...,...,...,...,...,...,...
13034,2019.03.31,,[암호화폐뉴스] 31일 가상화폐 대표 종목 상승… 비트코인 23시 17분 기준 시...,,CCTV뉴스,http://www.cctvnews.co.kr
13035,2019.03.31,2019-03-31 23:31:00,"[가상화폐 뉴스] 03월 31일 23시 30분 비트코인(1.15%), 질리카(7.4...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
13036,2019.03.31,,[암호화폐 시황] 비트코인 0.06%↑ 비트코인캐시 0.53%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
13037,2019.03.31,,"[세계 거래소별 가상화폐 시황] 31일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr


In [57]:
result = pd.concat([df1, df2, df3], ignore_index=True)
result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
result

,date,time,title,content,press,link
0,2021.07.01,,"뉴욕증시 비트코인 휘청, 국채금리 급락 델타 변이 비상사태 선언+ 고용보고서...",,글로벌이코노믹,http://www.g-enews.com/
1,2021.07.01,,암호화폐 ‘큰손’ 노보그라츠 이번엔 블록체인 기반 가상 부동산서 ‘돈 냄새...,,글로벌이코노믹,http://www.g-enews.com/
2,2021.07.01,,전문가가 보는 2021년 상반기 비트코인 롤러코스터 행보에서 보는 결정적 약점...,,글로벌이코노믹,http://www.g-enews.com/
3,2021.07.01,,"[스타 사업가(10)]래퍼 제이-Z, NFT 영역 진입…블록체인 및 스마트 계약",,글로벌이코노믹,http://www.g-enews.com/
4,2021.07.01,2021-07-01 03:01:06,[경향의 눈] “억만장자는 지구를 떠나라”는 경고,\n\t\t\t[경향신문] 세계 최고 부자 제프 베이조스가 이달 20일 우주여행을 ...,A26면 TOP,https://n.news.naver.com/mnews/article/032/000...
...,...,...,...,...,...,...
11124,2021.08.31,,"[글로벌 규제 동향] 슬로베니아, 암호화폐 사용·판매에 10% 세금 부과 추진 ...",,코인리더스,http://coinreaders.com
11125,2021.08.31,,"엘살바도르 의회, 1.5억달러 규모 비트코인 신탁 승인...""달러와의 태환성 보...",,코인리더스,http://coinreaders.com
11126,2021.08.31,,비트코인 4만8000달러 아래서 고전하는 가운데 도지코인 50일 이동평균 수준 ...,,글로벌이코노믹,http://www.g-enews.com/
11127,2021.08.31,,"기관투자자들의 GBTC 매입 본격화?...""GBTC 락업 해제 BTC 급락 우려, ...",,코인리더스,http://coinreaders.com


In [58]:
result.to_csv('비트코인_20210701_20210831.csv', encoding='utf-8-sig', index=False)